In [55]:
# Pkg.add("GraphPlot")
using JuMP, Gurobi
using LightGraphs
using GraphPlot
const gurobi_env = Gurobi.Env()
function plotGraph(edge)
    global N, origin, destination
#     n = maximum(edge)
#     println(n)
    g = SimpleDiGraph(destination)
    label1 = collect(1:destination)
    for i = 1:length(edge[:,1])
        add_edge!(g, edge[i,1], edge[i,2]);
    end
    gplot(g, layout=circular_layout, nodelabel=label1)
#     kruskal_mst(g, ones(length(edge[:,1])))
#     println(is_strongly_connected(g))
end
function gx_bound(c, c_g, edge, source_sink)
    global  N, origin, destination
    #println(f,"Current CELL's LB = ", c_L)
    #println(f,"Current CELL's UB = ", c_U)
    #println(f, "Current interdiction x = ", x_now)
    if source_sink == true
        start_node = edge[:,1]
        end_node = edge[:,2]
        origin = 1
        destination = N
    else 
        start_node = edge[:,2]
        end_node = edge[:,1]
        origin = N
        destination = 1
    end
    no_node = max(maximum(start_node), maximum(end_node) )
    no_link = length(start_node)


    function getShortestX(state, start_node, end_node, origin, destination)
        _x = zeros(Int, length(start_node))
        _path = enumerate_paths(state, destination)

        for i=1:length(_path)-1
            _start = _path[i]
            _end = _path[i+1]

            for j=1:length(start_node)
                if start_node[j]==_start && end_node[j]==_end
                _x[j] = 1
                break
                end
            end

        end
        _x
    end


    graph = Graph(no_node)
    distmx = Inf*ones(no_node, no_node)

    # Adding links to the graph
    for i=1:no_link
        add_edge!(graph, start_node[i], end_node[i])
        distmx[start_node[i], end_node[i]] = c_g[i]
    end

    # Run Dijkstra's Algorithm from the origin node to all nodes
    state = dijkstra_shortest_paths(graph, origin, distmx)
    label = state.dists
    pred = state.parents
    b_arc = ""
    
    for i = 1: length(state.parents)
        if state.parents[i] != 0 
            b_arc = string(b_arc, "(", state.parents[i], ",", i, ")")
        end
    end
    
    # Retrieving the shortest path
    path = enumerate_paths(state, destination)
    
    #parents = LightGraphs.DijkstraState(state, destination)

    # Retrieving the 'x' variable in a 0-1 vector
    y = getShortestX(state, start_node, end_node, origin, destination)
    #println(f,"y vector:", y)
    
    gx = sum(c_g[i]*y[i] for i = 1:no_link)    
    SP = sum(c[i]*y[i] for i = 1:length(c))
    T = Int64[]
    for i = 1:Len
        if pred[edge[i,2]] == edge[i,1]
            push!(T, 1)
        else
            push!(T, 0)
        end
    end
    
    origin = 1
    destination = N
#     println("Minimum Spanning Tree: ", T)
    #println("Shortest path y = ", y)
    #println("Indices of shortest path (edge) = ", y_index)
    #println("Nodes visited =", path)
#     println("Minimum Spanning Tree =", pred)
#     println("Node label = " ,label)

    return y, gx, SP, label
end

function x_disconnect(arcs, b_x, d_x, origin, destination, y_init)
    is_connected = true
    numArcs = length(arcs[:,1])
    m = Model(() -> Gurobi.Optimizer(gurobi_env))
    set_optimizer_attribute(m, "OutputFlag", 0)
    @variable(m, u >= 0)#[1:numArcs], Bin)
    @variable(m, x[1:numArcs], Bin)
    @constraint(m, sum(d_x[a]*x[a] for a=1:numArcs) <= b_x)
    @constraint(m, u >= 1 - sum(x[a]*y_init[a] for a = 1:numArcs))
#     #Source node: 
#     outArcs = findall(arcs[:,1].== origin)
#     @constraint(m, sum(u[a] for a in outArcs) == 1)
    #Plus Obj Func
    @objective(m, Min, u)
    
    terminate = false
    while terminate == false
        optimize!(m)
        
        u_now =JuMP.value.(u)
        x_now =JuMP.value.(x)
        print("u = ", u_now,";\t")
        print("x = ", findall(x_now.>=0.999), ";\t")
        if u_now >= 0.999
            terminate = true
            is_connected = true
        else
            
            intArcs = findall(x_now .>= 0.999)
        #     arcs_now = arcs[notIntArc, :]
            c = ones(numArcs)
            for a in intArcs
                c[a] = 1000
            end
            y, gx, SP, label1 = gx_bound(c, c, arcs, true)
            println("SP cost ", gx)
            if gx > 999
                terminate = true
                is_connected = false
            else 
                arcsinPath = findall(y.>=0.999)
                @constraint(m, u >= 1-sum(x[a] for a in arcsinPath))
            end  
        end
#             if gx > 999
#                 terminate = true
#                 is_connected = false
#             else 
#                 is_connected = true
#             end
        
    end
    println(m)
#     #Other nodes:
#     for i = (origin + 1):(destination-1)
#         outArcs = findall(arcs[:,1].== i)
#         inArcs = findall(arcs[:,2].== i)
#         @constraint(m, sum(u[a] for a in outArcs) - sum(u[a] for a in inArcs)==0)
#     end
    
#     #x-constraint:
#     for a = 1:numArcs
#         @constraint(m, u[a] <= 1-x[a])
#     end
    
#     optimize!(m)
#     println(m)
#     if termination_status(m) == MOI.OPTIMAL
#         is_connected = true
#         println("Obj = ", JuMP.objective_value.(m))
#     else
#         is_connected = false
#     end
    
    return is_connected
end

# function print2File()

# edge = [1 4; 2 6; 2 8; 2 10; 2 11; 2 15; 3 11; 4 13; 5 3; 5 13; 6 8; 7 2; 7 11; 7 15; 8 4; 8 7; 8 12; 8 15; 9 2; 9 6; 9 8; 9 13; 10 11; 11 10; 12 2; 12 5; 12 6; 12 11; 13 2; 13 7; 13 14; 14 5; 14 9; 14 13]
# edge = [1 2; 1 3; 3 2]
# global Len = length(edge[:,1])
# c = ones(Len)
# c_g = ones(Len)
# global origin = 1
# global destination = 3

Academic license - for non-commercial use only - expires 2022-12-04


x_disconnect (generic function with 2 methods)

In [56]:
ins = 1#ARGS[1] #:10
dataSet = "N10_d10_"
include("./"*dataSet*string(ins)*".jl")
numArcs =length(arcs[:,1])
c = ones(numArcs)
y_init, gx, SP, label1 = gx_bound(c, c, arcs, true)
is_connected = x_disconnect(arcs, b_x, d_x, origin, destination, y_init)
println(is_connected)
# println(arcs)
# println(d_y)
# mA= [16,17,18, 5, 19,8]
# println(arcs[mA,:])
# println(d_y[mA])
# plotGraph(arcs)

#[3 7; 5 10]
# y, gx, SP = gx_bound(c, c_g, edge)

u = 0.0;	x = [2];	SP cost 3.0
u = 0.0;	x = [2, 15];	SP cost 6.0
u = 1.0;	x = Int64[];	Min u
Subject to
 u + x[2] >= 1.0
 u + x[1] + x[4] + x[15] >= 1.0
 u + x[1] + x[5] + x[6] + x[10] + x[13] + x[16] >= 1.0
 6 x[1] + x[2] + 4 x[3] + 7 x[4] + 3 x[5] + 7 x[6] + 4 x[7] + x[8] + 3 x[9] + 7 x[10] + 6 x[11] + 6 x[12] + 2 x[13] + 5 x[14] + 3 x[15] + 9 x[16] + 10 x[17] <= 5.0
 u >= 0.0
 x[1] binary
 x[2] binary
 x[3] binary
 x[4] binary
 x[5] binary
 x[6] binary
 x[7] binary
 x[8] binary
 x[9] binary
 x[10] binary
 x[11] binary
 x[12] binary
 x[13] binary
 x[14] binary
 x[15] binary
 x[16] binary
 x[17] binary

true


In [61]:
P = [1,5,6,10,13,16]
println(arcs[P,:])
println(d_y[P])
setdiff(P, [1,5])

[1 3; 3 9; 4 10; 6 4; 7 6; 9 7]
[9.0, 2.0, 2.0, 3.0, 1.0, 2.0]


4-element Array{Int64,1}:
  6
 10
 13
 16

In [53]:

# Pkg.add("GraphPlot")
# num = "14"
# edge = Int64[]
global edge = Array{Int64}(undef, 0, 2)
global N = 10
global origin = 1
global destination = N
global numArcs = 0
global b_x = 5
global b_y = 10
global d_x = []
global d_y = []

# global uncNum = 0.1 # 0.2
for num = 1:5 #0
    global edge, N, origin, destination, numArcs, uncNum, d_x, d_y
    print("",num,"...")
    stop_criterion = false
    density = 0.1
    times = 0
    while stop_criterion == false
#         if times > 5
#             break
#         end
        edge = Array{Int64}(undef, 0, 2)
        head = zeros(N)
        tail = zeros(N)
            for u = 1 : N-1
                for v = 2 : N
                    x = rand()
                    if x <= density && u != v 
                        edge = vcat(edge,[u v])
                    end
                end
            end
#         println("",edge)
        numArcs = length(edge[:,1])
        c_temp = ones(numArcs)
        
        #Verify there is a path from source to every node and from every node to sink
        try
#             print("x")
            y_init, gx, SP, label1 = gx_bound(c_temp, c_temp, edge, true)
    #         println("label1 = ", label1)
            y_init2, gx, SP, label2 = gx_bound(c_temp, c_temp, edge, false)
            if isfinite(maximum(label1))==true && isfinite(maximum(label2)) == true
#                 stop_criterion = true
#                 println("numArcs = ", numArcs)
                d_x = Float64[]
                d_y = Float64[]
                for i = 1 : numArcs
                    x = rand(1:10)
                    y = rand(1:10)
                    push!(d_x, x)
                    push!(d_y, y)
                end
                stop_criterion = x_disconnect(edge, b_x, d_x, origin, destination, y_init)
                print(stop_criterion," ")
            end
        catch
            stop_criterion = false
        end
        
        
    end
    println(edge)
    println(length(edge[:,1]))
    plotGraph(edge)
    
    #Generate q-values as detection probability in (0,1) , 3 decimals
    q = zeros(numArcs)
    p = rand(1:999,numArcs) #Float64[]
    for i = 1 : numArcs
#         p = 
        q[i] = rand(p[i]:999)#/1000
#         if x<0
#             x=0
#         end
#         push!(q, x)
    end
    q = q./1000
    p = p./1000
    println("p = ", p)
    println("q = ", q)
    println("y")
#     println(cL)
#     println(cU)
    # println(myindex)
    origin = 1
    destination = N
    
#     println("Here:", Len) 
    y, gx, SP, label = gx_bound(q, q, edge, true)
    # println(gx)
#     println("HERE")
    
    # println(d)
    d_name = convert(Int64, density*100) #int(density*100)
    outfile = "./N"*string(N)*"_d"*string(d_name)*"_"*string(num)*".jl"
#     outfile = "/Users/din/Desktop/Research/GenerateTestSet/Test_"*string(num)*".jl"
    f = open(outfile, "w")
    println(f,"global arcs = ", edge) #length(df[:CELL]))
    println(f,"global d_x = ", d_x)
    println(f,"global b_x = ", b_x)
    println(f,"global d_y = ", d_y)
    println(f,"global b_y = ", b_y)
    println(f,"global p = ", p)
    println(f,"global q = ", q)
    println(f,"global origin = 1")
    println(f,"global destination = ", N)
    close(f)
end

1...u = 0.0;	x = [2];	SP cost 2.0
u = -0.0;	x = [1, 2];	SP cost 1000.0
false u = 0.0;	x = [1];	SP cost 4.0
u = 0.0;	x = [11];	SP cost 1003.0
false u = -0.0;	x = [1];	SP cost 1004.0
false u = 0.0;	x = [2];	SP cost 3.0
u = 0.0;	x = [2, 15];	SP cost 6.0
u = 1.0;	x = Int64[];	true [1 3; 1 10; 2 6; 3 8; 3 9; 4 10; 5 2; 5 7; 5 8; 6 4; 6 5; 6 8; 7 6; 7 8; 8 10; 9 7; 9 8]
17
p = [0.381, 0.401, 0.787, 0.933, 0.032, 0.91, 0.113, 0.737, 0.268, 0.924, 0.758, 0.337, 0.29, 0.765, 0.875, 0.088, 0.341]
q = [0.911, 0.544, 0.935, 0.935, 0.323, 0.978, 0.695, 0.986, 0.747, 0.978, 0.954, 0.909, 0.792, 0.894, 0.888, 0.599, 0.859]
y
2...u = -0.0;	x = [7];	SP cost 6.0
u = 1.0;	x = [5];	true [1 8; 2 7; 2 9; 2 10; 3 2; 3 6; 4 10; 5 7; 6 2; 7 3; 8 4; 8 5; 9 3; 9 8]
14
p = [0.508, 0.821, 0.748, 0.046, 0.867, 0.688, 0.559, 0.045, 0.016, 0.031, 0.488, 0.91, 0.795, 0.319]
q = [0.881, 0.906, 0.959, 0.237, 0.975, 0.81, 0.711, 0.442, 0.507, 0.7, 0.976, 0.996, 0.817, 0.741]
y
3...u = 1.0;	x = Int64[];	true [1 4; 1 9; 2 

In [ ]:
import Statistics
set_ins = rand(1:200,20)
temp_ = rand(20)
println(Statistics.mean(temp_[1:5]))
# average()
 for num in set_ins
#     include("./N50_Ins_"*string(num)*".jl")
#     A = length(edge)
#     println(num, ". min cL", minimum(cL_orig), "; max M ", maximum(M_orig[findall(M_orig.<20)]), "; max d ", maximum(d))
#     println("density = ", A/destination/(destination-1))
# end

In [ ]:
edge = [1 2; 2 3; 1 3]
c_g = [1, 1.5, 10]
function dijkstra(c_g, edge)
    no_node = maximum(edge)
    no_link = length(edge[:,1])
    start_node = edge[:,1]
    end_node = edge[:,2]
    
    graph = SimpleDiGraph(no_node)
    distmx = Inf*ones(no_node, no_node)

# Adding links to the graph
    for i=1:no_link
        add_edge!(graph, start_node[i], end_node[i])
        distmx[start_node[i], end_node[i]] = c_g[i]
    end
    A = dijkstra_shortest_paths(graph, origin, distmx)
    return A
end
A = dijkstra(c_g, edge)

In [ ]:
#Generate New Files Unc Range /2 and *2 for each arc
# global edge = Array{Int64}(undef, 0, 2)
# global N = 50 #100
# global origin = 1
# global destination = N
# global Len = 0 
function print2File(cL, cU, stat, myFile)
    println(stat)
    println(myFile)
    outfile = "./N50_"*stat*"UncRange_"*myFile*".jl"
    println(outfile)
#     outfile = "/Users/din/Desktop/Research/GenerateTestSet/Test_"*string(num)*".jl"
    f = open(outfile, "w")
    println(f,"edge = ", edge) #length(df[:CELL]))
    println(f,"cL_orig = ", cL)
    println(f,"cU_orig = ", cU)
    println(f,"d = ", d)
    println(f,"Len = length(cL_orig)")

    println(f,"c_orig = 0.5*(cL_orig+cU_orig)")

    #STARTING SOLUTION:
    println(f,"yy = zeros(Len)")
    println(f,"yy = ", yy)
    println(f,"SP_init = sum(yy[i]*c_orig[i] for i = 1:Len)")
    println(f,"p = [1.0]")
    println(f,"g = [SP_init]")
    println(f,"h = [0.0]")

    println(f,"origin = 1")
    println(f,"destination = 50")

    println(f,"last_node = maximum(edge)")
    println(f,"all_nodes = collect(1:last_node)")

    println(f,"M_orig = zeros(Len)")

    println(f,"for i = 1:Len")
    println(f,"    M_orig[i] = cU_orig[i] - cL_orig[i]")
    println(f,"end")


    println(f,"delta1 = ", delta1)
    println(f,"delta2 = ", delta2)
    println(f, "b = ", b) #rand(1:b_max))
    
#     println(f, )
    println(f,"last_node = maximum(edge)")
    println(f,"β = ", β)
    close(f)
end
function divideby2(cL, cU, M, myFile)
    c_bar = 0.5.*(cL + cU)
    M = 0.25.*M
    cL = c_bar - M
    cU = c_bar + M
    print2File(cL, cU, "Half", myFile)
#     println("\ncL = ", cL[M.>0])
#     println("cU = ", cU[M.>0])
end
function multby2(cL, cU, M, myFile)
    c_bar = 0.5.*(cL + cU)
#     M = 0.25.*M
    cL = c_bar - M
    cU = c_bar + M
    A_neg = findall(cL.<0)
#     println("\nA_neg = ", A_neg)
    for a in A_neg
        cU[a] = cU[a]-cL[a]
        cL[a] = 0
    end
    print2File(cL, cU, "Double", myFile)
#     println(cL[A_neg])
#     println("\ncL = ", cL[M.>0])
#     println("cU = ", cU[M.>0])
end

for ins in [16,35,41,43,56,79,84,110,118,142]
    myFile = string(ins)#*"_lastr"
    include("./N50/N50_"*myFile*".jl")
#     println("cL = ", cL_orig)
#     println("M_orig = ", M_orig)
    uncA = findall(M_orig.>0)
    println("density = ", length(uncA)/length(edge[:,1]))
#     println("uncA (", length(uncA),") = ", uncA)
#     println("cL_orig = ", cL_orig[M_orig.>0])
#     println("cU_orig = ", cU_orig[M_orig.>0])
    
    divideby2(cL_orig, cU_orig, M_orig,myFile)
    multby2(cL_orig, cU_orig, M_orig, myFile)
#     println("M_orig = ", M_orig[M_orig.>0])
end

In [ ]:
#Change budget to b=4 and b=10 for each set:
function updateBudgetb(myFile,b_new)
    println("b_new = ", b_new)
    println(myFile)
    outfile = "./N50_b"*string(b_new)*"_"*myFile*".jl"
    println(outfile)
#     outfile = "/Users/din/Desktop/Research/GenerateTestSet/Test_"*string(num)*".jl"
    f = open(outfile, "w")
    println(f,"edge = ", edge) #length(df[:CELL]))
    println(f,"cL_orig = ", cL_orig)
    println(f,"cU_orig = ", cU_orig)
    println(f,"d = ", d)
    println(f,"Len = length(cL_orig)")

    println(f,"c_orig = 0.5*(cL_orig+cU_orig)")

    #STARTING SOLUTION:
    println(f,"yy = zeros(Len)")
    println(f,"yy = ", yy)
    println(f,"SP_init = sum(yy[i]*c_orig[i] for i = 1:Len)")
    println(f,"p = [1.0]")
    println(f,"g = [SP_init]")
    println(f,"h = [0.0]")

    println(f,"origin = ", origin)
    println(f,"destination = ", destination)

    println(f,"last_node = maximum(edge)")
    println(f,"all_nodes = collect(1:last_node)")

    println(f,"M_orig = zeros(Len)")

    println(f,"for i = 1:Len")
    println(f,"    M_orig[i] = cU_orig[i] - cL_orig[i]")
    println(f,"end")


    println(f,"delta1 = ", delta1)
    println(f,"delta2 = ", delta2)
    println(f, "b = ", b_new) #rand(1:b_max))
    
#     println(f, )
    println(f,"last_node = maximum(edge)")
    println(f,"β = ", β)
    close(f)
end

for ins in [16,35,41,43,56,79,84,110,118,142]
    myFile = string(ins)#*"_lastr"
    include("./N50/N50_"*myFile*".jl")
#     println("density = ", length(uncA)/length(edge[:,1]))
    b_new = 4
    updateBudgetb(myFile, b_new)
    b_new = 10
    updateBudgetb(myFile, b_new)
end

In [ ]:

set = rand(1:200, 10)
for ins in set # [16,35,41,43,56,79,84,110,118,142]
    myFile = string(ins)#*"_lastr"
    dataSet = "N100"
    include("./"*dataSet*"/"*dataSet*"_"*myFile*".jl")
#     println("cL = ", cL_orig)
#     println("M_orig = ", M_orig)
    uncA = findall(M_orig.>0)
    println(ins," .UncNum = ", length(uncA)/length(edge[:,1]))
    println("\t density = ", length(d)/destination/(destination-1))
    println("\t maxM = ", maximum(M_orig))
    println("\t min c = ", minimum(cL_orig))
    println("\t b = ", b)
end

In [51]:
rand(1:999,10)

10-element Array{Int64,1}:
 989
  31
 281
 307
 861
 249
 910
  24
 356
 181